In [4]:
!pip3 install -U pip
!pip3 install -U setuptools wheel

# CPU version of pytorch has smaller footprint - see installation instructions in
# pytorch documentation - https://pytorch.org/get-started/locally/
!pip3 install torch==1.12+cpu torchvision==0.13.0+cpu torchtext==0.13.0 -f https://download.pytorch.org/whl/cpu/torch_stable.html

!pip3 install autogluon

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 353.9 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 22.1.1
    Uninstalling pip-22.1.1:
      Successfully uninstalled pip-22.1.1
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-cor

  Using cached openmim-0.2.1-py2.py3-none-any.whl (49 kB)
  Using cached lightgbm-3.3.3.tar.gz (1.5 MB)
  Preparing metadata (setup.py) ... done
  Using cached fastai-2.7.10-py3-none-any.whl (240 kB)
  Using cached catboost-1.1.1-cp39-none-macosx_10_6_universal2.whl (22.0 MB)
  Using cached tbats-1.1.1-py3-none-any.whl (43 kB)
  Using cached gluoncv-0.10.5.post0-py2.py3-none-any.whl (1.3 MB)
  Using cached albumentations-1.1.0-py3-none-any.whl (102 kB)
  Using cached datasets-2.7.1-py3-none-any.whl (451 kB)
  Using cached spacy-3.4.3-cp39-cp39-macosx_11_0_arm64.whl (6.5 MB)


  Using cached tensorboard-2.11.0-py3-none-any.whl (6.0 MB)


  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [80 lines of output]
      INFO:root:running bdist_wheel
      /opt/homebrew/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
        warnings.warn(
      INFO:root:running build
      INFO:root:running build_py
      INFO:root:creating build
      INFO:root:creating build/lib
      INFO:root:creating build/lib/lightgbm
      INFO:root:copying lightgbm/callback.py -> build/lib/lightgbm
      INFO:root:copying lightgbm/compat.py -> build/lib/lightgbm
      INFO:root:copying lightgbm/plotting.py -> build/lib/lightgbm
      INFO:root:copying lightgbm/__init__.py -> build/lib/lightgbm
      INFO:root:copying lightgbm/engine.py -> build/lib/lightgbm
      INFO:root:copying lightgbm/dask.py -> build/lib/lightgbm
      INFO:root:copying lightgbm/bas

In [5]:
import pandas as pd
import numpy as np

from autogluon.tabular import TabularPredictor
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config
from autogluon.multimodal import MultiModalPredictor

ModuleNotFoundError: No module named 'autogluon'

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [ ]:
check = []
check1 = []
temp = train['Utterance'].apply(len)
train['length'] = temp

for i in range(len(train)):
    if train['length'].loc[i] <= 2:
#         print(train['Utterance'].loc[i])
        if train['Utterance'].iloc[i] != 'Hi' and train['Utterance'].iloc[i] != 'No':
            check.append(train.index[i])
            check1.append(train['Utterance'].iloc[i])
train = train.drop(labels = check, axis = 0)
train

In [ ]:
hyperparameters = get_hyperparameter_config('multimodal')
hyperparameters

In [ ]:
hyperparameters['XGB'] = {'tree_method':'gpu_hist', 'gpu_id':0}
hyperparameters['AG_TEXT_NN'] = {'presets': 'best_quality'}
hyperparameters.pop('VW')
hyperparameters.pop('NN_TORCH')

In [ ]:
hyperparameters

In [ ]:
hyperparameters = {
    "model.names": ["hf_text", "categorical_mlp", "numerical_mlp", "fusion_mlp"],
    "model.hf_text.checkpoint_name": "tae898/emoberta-base",
    }

In [ ]:
train['Dialogue_ID'] = train['Dialogue_ID'].astype('category')
test['Dialogue_ID'] = test['Dialogue_ID'].astype('category')
train = train.drop('ID', axis=1)
test = test.drop('ID', axis=1)

In [ ]:
predictor = MultiModalPredictor(label='Target', problem_type='multiclass', eval_metric='log_loss')
predictor.fit(train, hyperparameters=hyperparameters)

In [ ]:
predictions = predictor.predict(test)

In [ ]:
sample_submission['Target'] = predictions

In [ ]:
sample_submission.to_csv('./submit3.csv',index=False)

In [ ]:
from google.colab import files
files.download("submit3.csv")